In [ ]:
import pandas as pd
import json
import numpy as np
import os
import re
from pathlib import Path
from azure.core.credentials import AzureKeyCredential
from azure.ai.documentintelligence import DocumentIntelligenceClient
from azure.ai.documentintelligence.models import AnalyzeDocumentRequest

endpoint = "https://ocrzagoprice2.cognitiveservices.azure.com/"
key = "B9UhzXXGyeA99K4MJUTdQLtwgmdkCwA1cwrpaFzIYS79fpOSWj0MJQQJ99BEACYeBjFXJ3w3AAALACOGFMZG"

document_intelligence_client  = DocumentIntelligenceClient(
    endpoint=endpoint, credential=AzureKeyCredential(key)
)

def column_contains_keyword(series):
    return series.astype(str).str.lower().apply(
        lambda val: any(kw in val for kw in keywords)
    ).any()




In [ ]:
# First try
formUrl = "https://raw.githubusercontent.com/ezagoc/prisions_capacity/main/raw/2000/2000_01_CE.pdf"

poller = document_intelligence_client.begin_analyze_document(
    "prebuilt-layout", AnalyzeDocumentRequest(url_source=formUrl)
)
result = poller.result()

df_final = pd.DataFrame()
for idx, table in enumerate(result.tables):
    rows = table['rowCount']
    cols = table['columnCount']
    grid = [[np.nan for _ in range(cols)] for _ in range(rows)]

    for cell in table['cells']:
        r = cell['rowIndex']
        c = cell['columnIndex']
        val = cell.get('content', '').strip()
        if val:
            grid[r][c] = val

    df = pd.DataFrame(grid)
    df = df[~df.apply(lambda row: row.isna().all(), axis=1)]
    new_column_names = {old_name: f"col{i+1}" for i, old_name in enumerate(df.columns)}
    df.rename(columns=new_column_names, inplace=True)
    for col in range(1, df.shape[1]):
        if pd.isna(df.iat[0, col]):
            df.iat[0, col] = df.iat[0, col - 1]
    
    # Define keywords to search for
    keywords = ["TOTAL", "Capacidad", "Subtotal"]

    # Normalize to lowercase for matching
    keywords = [k.lower() for k in keywords]

    # Filter: keep only rows that match any keyword
    # Identify columns to keep
    columns_to_keep = [col for col in df.columns if column_contains_keyword(df[col])]
    actual = list(set(['col1'] + columns_to_keep))
    print(actual)
    # Filter the DataFrame
    df = df[actual]

    new_columns = []

    for i, val in enumerate(df.iloc[0]):
        if isinstance(val, str) and 'Centro' in val:
            new_columns.append('center_name')
        elif isinstance(val, str) and 'Capacidad' in val:
            new_columns.append('capacity')
        elif isinstance(val, str) and 'FEDERAL' in val:
            new_columns.append('federal')
        elif isinstance(val, str) and 'COMUN' in val:
            new_columns.append('comun')
        else:
            new_columns.append(f'total')  # fallback/default name

    # Set the new column names and drop the first row
    df.columns = new_columns
    df = df[1:].reset_index(drop=True)

    df_final = pd.concat([df_final, df])

df_final = df_final[~df_final['center_name'].isnull()]
df_final['month'] = 1
df_final['year'] = 2000

df_final.to_excel('../../../data/00-map/capacity/raw/2000/capacity_1.xlsx', index = False)

In [ ]:
# First OCR every month

year = 2000
for i in range(2, 13):
    ii = f"{i:02d}"
    formUrl = f"https://raw.githubusercontent.com/ezagoc/prisions_capacity/main/raw/{year}/{year}_{ii}_CE.pdf"
    print(formUrl)
    poller = document_intelligence_client.begin_analyze_document(
        "prebuilt-layout", AnalyzeDocumentRequest(url_source=formUrl))
    result = poller.result()
    for idx, table in enumerate(result.tables):
        rows = table['rowCount']
        cols = table['columnCount']
        grid = [[np.nan for _ in range(cols)] for _ in range(rows)]

        for cell in table['cells']:
            r = cell['rowIndex']
            c = cell['columnIndex']
            val = cell.get('content', '').strip()
            if val:
                grid[r][c] = val

        df = pd.DataFrame(grid)
        df = df[~df.apply(lambda row: row.isna().all(), axis=1)]
        new_column_names = {old_name: f"col{i+1}" for i, old_name in enumerate(df.columns)}
        df.rename(columns=new_column_names, inplace=True)

        folder = Path(f"../../../data/00-map/capacity/raw/2000/tables{ii}")
        folder.mkdir(parents=True, exist_ok=True)

        df.to_parquet(f"../../../data/00-map/capacity/raw/2000/tables{ii}/table_{idx}.parquet.gzip", 
                      index = False, compression = 'gzip')



keywords = ["TOTAL", "Capacidad", "Subtotal"]
keywords = [k.lower() for k in keywords]

for e in range(2, 13): 
    print(e)
    ii = f"{e:02d}"
    table_path = f"../../../data/00-map/capacity/raw/2000/tables{ii}"
    files = [f for f in os.listdir(table_path) if f.endswith(".gzip")]
    files_sorted = sorted(files, key=lambda x: int(re.search(r'(\d+)', x).group()))
    df_final = pd.DataFrame()

    for table in files_sorted:
        df = pd.read_parquet(f'{table_path}/{table}')
        for col in range(1, df.shape[1]):
            if pd.isna(df.iat[0, col]):
                df.iat[0, col] = df.iat[0, col - 1]
    
    

    # Filter: keep only rows that match any keyword
    # Identify columns to keep
        columns_to_keep = [col for col in df.columns if column_contains_keyword(df[col])]
        actual = list(set(['col1'] + columns_to_keep))
        print(actual)
    # Filter the DataFrame
        df = df[actual]

        new_columns = []

        for i, val in enumerate(df.iloc[0]):
            if isinstance(val, str) and 'Centro' in val:
                new_columns.append('center_name')
            elif isinstance(val, str) and 'Capacidad' in val:
                new_columns.append('capacity')
            elif isinstance(val, str) and 'FEDERAL' in val:
                new_columns.append('federal')
            elif isinstance(val, str) and 'COMUN' in val:
                new_columns.append('comun')
            else:
                new_columns.append(f'total')  # fallback/default name

    # Set the new column names and drop the first row
        df.columns = new_columns
        df = df.loc[:, ~df.columns.duplicated()]
        df = df[1:].reset_index(drop=True)

        df_final = pd.concat([df_final, df])
        df_final = df_final[~df_final['center_name'].isnull()]
        df_final['month'] = e
        df_final['year'] = 2000

        df_final.to_excel(f'../../../data/00-map/capacity/raw/2000/capacity_{e}.xlsx', index = False)


https://raw.githubusercontent.com/ezagoc/prisions_capacity/main/raw/2000/2000_03_CE.pdf
https://raw.githubusercontent.com/ezagoc/prisions_capacity/main/raw/2000/2000_04_CE.pdf
https://raw.githubusercontent.com/ezagoc/prisions_capacity/main/raw/2000/2000_05_CE.pdf
https://raw.githubusercontent.com/ezagoc/prisions_capacity/main/raw/2000/2000_06_CE.pdf
https://raw.githubusercontent.com/ezagoc/prisions_capacity/main/raw/2000/2000_07_CE.pdf
https://raw.githubusercontent.com/ezagoc/prisions_capacity/main/raw/2000/2000_08_CE.pdf
https://raw.githubusercontent.com/ezagoc/prisions_capacity/main/raw/2000/2000_09_CE.pdf
https://raw.githubusercontent.com/ezagoc/prisions_capacity/main/raw/2000/2000_10_CE.pdf
https://raw.githubusercontent.com/ezagoc/prisions_capacity/main/raw/2000/2000_11_CE.pdf
https://raw.githubusercontent.com/ezagoc/prisions_capacity/main/raw/2000/2000_12_CE.pdf


In [149]:
# Function: Takes the year and month, OCRs it and gives you the result (and saves the parquet files for later)
from unidecode import unidecode

keywords = ["TOTAL", "Capacidad", "Subtotal", 'Espacios']
keywords = [k.lower() for k in keywords]

def ocr_parquet(month, year, to_parquet = True):
    month_string = f"{month:02d}"
    formUrl = f"https://raw.githubusercontent.com/ezagoc/prisions_capacity/main/raw/{year}/{year}_{month_string}_CE.pdf"
    print(formUrl)
    poller = document_intelligence_client.begin_analyze_document(
        "prebuilt-layout", AnalyzeDocumentRequest(url_source=formUrl))
    result = poller.result()

    if to_parquet:
        for idx, table in enumerate(result.tables):
            rows = table['rowCount']
            cols = table['columnCount']
            grid = [[np.nan for _ in range(cols)] for _ in range(rows)]

            for cell in table['cells']:
                r = cell['rowIndex']
                c = cell['columnIndex']
                val = cell.get('content', '').strip()
                if val:
                    grid[r][c] = val

            df = pd.DataFrame(grid)
            df = df[~df.apply(lambda row: row.isna().all(), axis=1)]
            new_column_names = {old_name: f"col{i+1}" for i, old_name in enumerate(df.columns)}
            df.rename(columns=new_column_names, inplace=True)

            folder = Path(f"../../../data/00-map/capacity/raw/{year}/tables{month_string}")
            folder.mkdir(parents=True, exist_ok=True)

            df.to_parquet(f"../../../data/00-map/capacity/raw/{year}/tables{month_string}/table_{idx}.parquet.gzip", 
                      index = False, compression = 'gzip')
        print(f'Parquets generated in path: {folder}')
    
    else:
        print('No parquet generated')
    return(result)


# Function that cleans the ocr output from the result object: 

def clean_ocr(result_ocr):
    df_final = pd.DataFrame()
    for idx, table in enumerate(result_ocr.tables):
        rows = table['rowCount']
        cols = table['columnCount']
        grid = [[np.nan for _ in range(cols)] for _ in range(rows)]

        for cell in table['cells']:
            r = cell['rowIndex']
            c = cell['columnIndex']
            val = cell.get('content', '').strip()
            if val:
                grid[r][c] = val

        df = pd.DataFrame(grid)
        df = df[~df.apply(lambda row: row.isna().all(), axis=1)]
        new_column_names = {old_name: f"col{i+1}" for i, old_name in enumerate(df.columns)}
        df.rename(columns=new_column_names, inplace=True)

    
        for col in range(1, df.shape[1]):
            if pd.isna(df.iat[0, col]):
                df.iat[0, col] = df.iat[0, col - 1]
    
    

        # Filter: keep only rows that match any keyword
        # Identify columns to keep
    
        columns_to_keep = [col for col in df.columns if column_contains_keyword(df[col])]
        actual = list(set(['col1'] + columns_to_keep))
        if len(actual) < 5:
            print('This file contains at least one missing column, check')
        # Filter the DataFrame
        df = df[actual]

        new_columns = []

        for i, val in enumerate(df.iloc[0]):
            if isinstance(val, str) and 'centro' in val.lower():
                new_columns.append('center_name')
            elif isinstance(val, str) and ('capacidad' in val.lower() or 'espacios' in val.lower()):
                new_columns.append('capacity')
            elif isinstance(val, str) and 'federal' in val.lower():
                new_columns.append('federal')
            elif isinstance(val, str) and 'comun' in unidecode(val).lower():
                new_columns.append('comun')
            else:
                new_columns.append(f'total')  # fallback/default name

    # Set the new column names and drop the first row
        df.columns = new_columns
        df = df.loc[:, ~df.columns.duplicated()]
        df = df[1:].reset_index(drop=True)

        df_final = pd.concat([df_final, df])
    return(df_final)

    

In [ ]:
# First try, verify that the ocr is the thing that is wrong, not the code
result1 = ocr_parquet(month = 12, year = 2003)

df_final = clean_ocr(result1)

df_final = df_final[~df_final['center_name'].isnull()]
df_final['month'] = 12
df_final['year'] = 2003

df_final.to_excel(f'../../../data/00-map/capacity/raw/{2003}/capacity_{12}.xlsx', 
                          index = False)

https://raw.githubusercontent.com/ezagoc/prisions_capacity/main/raw/2003/2003_12_CE.pdf
Parquets generated in path: ..\..\..\data\00-map\capacity\raw\2003\tables12


In [ ]:
# 2001 - 2002
for y in range(2001, 2003):
    for m in range(1, 13): 
        result1 = ocr_parquet(month = m, year = y)
        df_final = clean_ocr(result1)
        df_final = df_final[~df_final['center_name'].isnull()]
        df_final['month'] = m
        df_final['year'] = y

        df_final.to_excel(f'../../../data/00-map/capacity/raw/{y}/capacity_{m}.xlsx', 
                          index = False)

https://raw.githubusercontent.com/ezagoc/prisions_capacity/main/raw/2001/2001_01_CE.pdf
Parquets generated in path: ..\..\..\data\00-map\capacity\raw\2001\tables01
This file contains at least one missing column, check
This file contains at least one missing column, check
This file contains at least one missing column, check
This file contains at least one missing column, check
This file contains at least one missing column, check
This file contains at least one missing column, check
https://raw.githubusercontent.com/ezagoc/prisions_capacity/main/raw/2001/2001_02_CE.pdf
Parquets generated in path: ..\..\..\data\00-map\capacity\raw\2001\tables02
This file contains at least one missing column, check
This file contains at least one missing column, check
This file contains at least one missing column, check
This file contains at least one missing column, check
This file contains at least one missing column, check
This file contains at least one missing column, check
This file contains at le

In [ ]:
for y in range(2003, 2004):
    for m in range(1, 13): 
        result1 = ocr_parquet(month = m, year = y)
        df_final = clean_ocr(result1)
        df_final = df_final[~df_final['center_name'].isnull()]
        df_final['month'] = m
        df_final['year'] = y

        df_final.to_excel(f'../../../data/00-map/capacity/raw/{y}/capacity_{m}.xlsx', 
                          index = False)

https://raw.githubusercontent.com/ezagoc/prisions_capacity/main/raw/2003/2003_01_CE.pdf
Parquets generated in path: ..\..\..\data\00-map\capacity\raw\2003\tables01
This file contains at least one missing column, check
This file contains at least one missing column, check
https://raw.githubusercontent.com/ezagoc/prisions_capacity/main/raw/2003/2003_02_CE.pdf
Parquets generated in path: ..\..\..\data\00-map\capacity\raw\2003\tables02
https://raw.githubusercontent.com/ezagoc/prisions_capacity/main/raw/2003/2003_03_CE.pdf
Parquets generated in path: ..\..\..\data\00-map\capacity\raw\2003\tables03
This file contains at least one missing column, check
This file contains at least one missing column, check
This file contains at least one missing column, check
This file contains at least one missing column, check
This file contains at least one missing column, check
https://raw.githubusercontent.com/ezagoc/prisions_capacity/main/raw/2003/2003_04_CE.pdf
Parquets generated in path: ..\..\..\data\

KeyError: 'center_name'

In [152]:
for y in range(2004, 2005):
    for m in range(1, 13): 
        result1 = ocr_parquet(month = m, year = y)
        df_final = clean_ocr(result1)
        df_final = df_final[~df_final['center_name'].isnull()]
        df_final['month'] = m
        df_final['year'] = y

        df_final.to_excel(f'../../../data/00-map/capacity/raw/{y}/capacity_{m}.xlsx', 
                          index = False)

https://raw.githubusercontent.com/ezagoc/prisions_capacity/main/raw/2004/2004_01_CE.pdf
Parquets generated in path: ..\..\..\data\00-map\capacity\raw\2004\tables01
This file contains at least one missing column, check
This file contains at least one missing column, check
This file contains at least one missing column, check
This file contains at least one missing column, check
This file contains at least one missing column, check
This file contains at least one missing column, check
This file contains at least one missing column, check
This file contains at least one missing column, check
This file contains at least one missing column, check
This file contains at least one missing column, check
This file contains at least one missing column, check
https://raw.githubusercontent.com/ezagoc/prisions_capacity/main/raw/2004/2004_02_CE.pdf
Parquets generated in path: ..\..\..\data\00-map\capacity\raw\2004\tables02
This file contains at least one missing column, check
This file contains at le

In [153]:
for y in range(2005, 2006):
    for m in range(1, 13): 
        result1 = ocr_parquet(month = m, year = y)
        df_final = clean_ocr(result1)
        df_final = df_final[~df_final['center_name'].isnull()]
        df_final['month'] = m
        df_final['year'] = y

        df_final.to_excel(f'../../../data/00-map/capacity/raw/{y}/capacity_{m}.xlsx', 
                          index = False)

https://raw.githubusercontent.com/ezagoc/prisions_capacity/main/raw/2005/2005_01_CE.pdf
Parquets generated in path: ..\..\..\data\00-map\capacity\raw\2005\tables01
This file contains at least one missing column, check
This file contains at least one missing column, check
This file contains at least one missing column, check
This file contains at least one missing column, check
This file contains at least one missing column, check
This file contains at least one missing column, check
This file contains at least one missing column, check
This file contains at least one missing column, check
This file contains at least one missing column, check
This file contains at least one missing column, check
This file contains at least one missing column, check
This file contains at least one missing column, check
https://raw.githubusercontent.com/ezagoc/prisions_capacity/main/raw/2005/2005_02_CE.pdf
Parquets generated in path: ..\..\..\data\00-map\capacity\raw\2005\tables02
This file contains at le

In [141]:
# analyze tables

df = pd.read_parquet('../../../data/00-map/capacity/raw/2003/tables12/table_1.parquet.gzip')
df

,col1,col2,col3,col4,col5,col6,col7,col8,col9,col10,col11,col12,col13,col14,col15,col16,col17
0,ENTIDAD FEDERATIVA / CENTRO PENITENCIARIO FEDERAL,ESPACIOS,Sobrepoblación,None,FUERO COMÚN,None,None,None,None,None,FUERO FEDERAL,None,None,None,None,None,TOTAL
1,None,None,Absoluta,Relativa,Personas Procesadas,None,Personas Sentenciadas,None,Subtotal,%,Personas Procesadas,None,Personas Sentenciadas,None,Subtotal,%,None
2,None,None,None,None,H,M,H,M,None,None,H,M,H,M,None,None,None
3,CRS Número 9 Acapetahua,200,-200,-100.00%,None,None,None,None,0,0.00%,None,None,None,None,0\n:unselected:,0.00%,0
4,CRS Número 16 Ocosingo (El Encino),120,187,155.83%,120,None,187,None,307,100.00%,None,None,None,None,0\n:unselected:,0.00%,307
5,CRS Número 12 Yajalón,132,2,1.52%,66,None,68,None,134,100.00%,None,None,None,None,0,0.00%,134
6,CRS Número 4 Femenil Tapachula,64,-4,-6.25%,None,34,None,21,55,91.67%,None,1,None,4,5,8.33%,60
7,CRS Número 7 Huixtla,40,90,225%,75,None,55,None,130,100.00%,None,None,None,None,0,0.00%,130
8,Centro Estatal Preventivo Numero 1 Chiapa de C...,146,-69,-47.26%,47,None,30,None,77,100.00%,None,None,None,None,0,0.00%,77
9,CRS Numero 15 Copainalá,120,-42,-35.00%,34,None,43,None,77,98.72%,None,None,1,None,1,1.28%,78


In [125]:
df_final = pd.DataFrame()
for idx, table in enumerate(result1.tables):
    rows = table['rowCount']
    cols = table['columnCount']
    grid = [[np.nan for _ in range(cols)] for _ in range(rows)]

    for cell in table['cells']:
        r = cell['rowIndex']
        c = cell['columnIndex']
        val = cell.get('content', '').strip()
        if val:
            grid[r][c] = val

    df = pd.DataFrame(grid)
    df = df[~df.apply(lambda row: row.isna().all(), axis=1)]
    new_column_names = {old_name: f"col{i+1}" for i, old_name in enumerate(df.columns)}
    df.rename(columns=new_column_names, inplace=True)

    
    for col in range(1, df.shape[1]):
        if pd.isna(df.iat[0, col]):
            df.iat[0, col] = df.iat[0, col - 1]
    
    

    # Filter: keep only rows that match any keyword
    # Identify columns to keep
    last_column_name = df.columns[-1]
    columns_to_keep = [col for col in df.columns if column_contains_keyword(df[col])]
    actual = list(set(['col1'] + columns_to_keep + [last_column_name]))
    print(actual)
    # Filter the DataFrame
    df = df[actual]

    new_columns = []

    for i, val in enumerate(df.iloc[0]):
        if isinstance(val, str) and 'Centro' in val:
            new_columns.append('center_name')
        elif isinstance(val, str) and 'Capacidad' in val:
            new_columns.append('capacity')
        elif isinstance(val, str) and 'FEDERAL' in val:
            new_columns.append('federal')
        elif isinstance(val, str) and 'COMUN' in val:
            new_columns.append('comun')
        else:
            new_columns.append(f'total')  # fallback/default name

    # Set the new column names and drop the first row
    df.columns = new_columns
    df = df.loc[:, ~df.columns.duplicated()]
    df = df[1:].reset_index(drop=True)

    df_final = pd.concat([df_final, df])
    df_final = df_final[~df_final['center_name'].isnull()]
    df_final['month'] = 1
    df_final['year'] = 2001

df_final.to_excel(f'../../../data/00-map/capacity/raw/2001/capacity_1.xlsx', index = False)


['col15', 'col17', 'col2', 'col1', 'col9']
['col2', 'col16', 'col1', 'col9', 'col14']
['col2', 'col16', 'col1', 'col9', 'col14']
['col17', 'col2', 'col16', 'col1', 'col9', 'col14']
['col2', 'col16', 'col1', 'col9', 'col14']
['col2', 'col16', 'col1', 'col9', 'col14']
['col2', 'col16', 'col1', 'col9', 'col14']
['col2', 'col16', 'col1', 'col9', 'col14']
['col2', 'col16', 'col1', 'col9', 'col14']
['col2', 'col16', 'col1', 'col9', 'col14']
['col2', 'col16', 'col1', 'col9', 'col14']
['col4', 'col1', 'col2']
['col1', 'col5']
['col8', 'col1', 'col5']
['col4', 'col1', 'col2']
['col1', 'col5']
['col8', 'col1', 'col5']
['col2', 'col16', 'col1', 'col9', 'col14']
['col2', 'col16', 'col1', 'col9', 'col14']
['col2', 'col16', 'col1', 'col9', 'col14']


In [ ]:
year = 2000
for i in range(2, 13):
    ii = f"{i:02d}"
    formUrl = f"https://raw.githubusercontent.com/ezagoc/prisions_capacity/main/raw/{year}/{year}_{ii}_CE.pdf"
    print(formUrl)
    poller = document_intelligence_client.begin_analyze_document(
        "prebuilt-layout", AnalyzeDocumentRequest(url_source=formUrl))
    result = poller.result()
    for idx, table in enumerate(result.tables):
        rows = table['rowCount']
        cols = table['columnCount']
        grid = [[np.nan for _ in range(cols)] for _ in range(rows)]

        for cell in table['cells']:
            r = cell['rowIndex']
            c = cell['columnIndex']
            val = cell.get('content', '').strip()
            if val:
                grid[r][c] = val

        df = pd.DataFrame(grid)
        df = df[~df.apply(lambda row: row.isna().all(), axis=1)]
        new_column_names = {old_name: f"col{i+1}" for i, old_name in enumerate(df.columns)}
        df.rename(columns=new_column_names, inplace=True)

        folder = Path(f"../../../data/00-map/capacity/raw/2000/tables{ii}")
        folder.mkdir(parents=True, exist_ok=True)

        df.to_parquet(f"../../../data/00-map/capacity/raw/2000/tables{ii}/table_{idx}.parquet.gzip", 
                      index = False, compression = 'gzip')